### Import necessary Libraries

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Load the file wonderland.txt and read the text in the file

In [0]:
filename = "/content/drive/My Drive/EIPSession2/SequenceModels/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [4]:
print(len(raw_text))

144437


In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))


In [6]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [7]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '0': 11, '3': 12, ':': 13, ';': 14, '?': 15, '[': 16, ']': 17, '_': 18, 'a': 19, 'b': 20, 'c': 21, 'd': 22, 'e': 23, 'f': 24, 'g': 25, 'h': 26, 'i': 27, 'j': 28, 'k': 29, 'l': 30, 'm': 31, 'n': 32, 'o': 33, 'p': 34, 'q': 35, 'r': 36, 's': 37, 't': 38, 'u': 39, 'v': 40, 'w': 41, 'x': 42, 'y': 43, 'z': 44, '\ufeff': 45}


In [0]:
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [0]:
no_punct_text = ""
for char in raw_text:
   if char not in punctuations:
       no_punct_text = no_punct_text + char

In [10]:
print(len(no_punct_text))

136117


In [0]:
no_punct_text=no_punct_text.replace("\n\n","\n")

In [12]:
len(no_punct_text)

135284

In [0]:
chars = sorted(list(set(no_punct_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [14]:
print(chars)

['\n', ' ', '0', '3', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [15]:
print(char_to_int)

{'\n': 0, ' ': 1, '0': 2, '3': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'j': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'w': 26, 'x': 27, 'y': 28, 'z': 29, '\ufeff': 30}


In [16]:
n_chars = len(no_punct_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  135284
Total Vocab:  31


In [0]:
corpus = no_punct_text.lower().split("\n") 

In [18]:
len(corpus)

2512

In [21]:
seq_length=len(corpus[0])
#len(corpus)
ctr =0
final_corpus = []
#final_corpus = [0 for i in range(len(corpus))]
for i in range(len(corpus)):
  #print(len(corpus[i]))
  if len(corpus[i]) != 0:
    final_corpus.append(corpus[i])
  if len(corpus[i]) >= seq_length:
    seq_length=len(corpus[i])  
  

print(seq_length)
print(len(final_corpus))

73
2481


In [0]:
print(final_corpus[0])

﻿alices adventures in wonderland


In [0]:
padded_corpus =[]
for i in range(len(final_corpus)):
  pad=seq_length-len(final_corpus[i])
  #print("Pad",pad)
  #print(final_corpus[i])
  padded =str(final_corpus[i]).ljust(seq_length," ")
  #print("afterpadding",len(padded))
  padded_corpus.append(padded)
  

In [0]:
clean_text=""
for i in range(len(padded_corpus)):
  clean_text = clean_text+padded_corpus[i]

In [0]:
chars = sorted(list(set(clean_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [25]:
n_chars = len(clean_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  181113
Total Vocab:  30


In [0]:
# prepare the dataset of input to output pairs encoded as integers
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = clean_text[i:i + seq_length]
	seq_out = clean_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
#cntr = 0
#for i in range(0, n_chars - seq_length, 1):
  #print("Hi")
  #if cntr < 20:
    #print(raw_text[i:i + seq_length])
    #print(len(raw_text[i:i + seq_length]))
    #print(raw_text[i + seq_length])
  #cntr = cntr +1
#print(cntr)
#n_patterns = len(dataX)
#print ("Total Patterns: ", n_patterns)

In [31]:
print ("Total Patterns: ", n_patterns)

Total Patterns:  181040


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [33]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True,dropout=0.1))
model.add(Dropout(0.1))
model.add(LSTM(256))
#model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

W0726 23:40:40.415006 140206028044160 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 23:40:40.432079 140206028044160 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 23:40:40.637341 140206028044160 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 23:40:40.648728 140206028044160 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Ins

In [0]:

# define the checkpoint
#filepath="/content/drive/My Drive/EIPSession2/SequenceModels/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
filepath="/content/drive/My Drive/EIPSession2/SequenceModels/weights-improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [0]:
#model.fit(X, y, epochs=5, batch_size=128, callbacks=callbacks_list)
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=2, batch_size=64, callbacks=callbacks_list)

W0726 23:41:19.478785 140206028044160 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 23:41:19.508822 140206028044160 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0726 23:41:19.657361 140206028044160 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/2
132160/181040 [====================>.........] - ETA: 3:01 - loss: 2.1147

In [0]:
# load the network weights
filename = "/content/drive/My Drive/EIPSession2/SequenceModels/weights-improvement.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
start = numpy.random.randint(0, len(dataX)-1)
print("Start is",start)
pattern = dataX[start]

print("Pattern is",pattern)
print("Pattern length is",len(pattern))
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(10):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Start is 11
Pattern is [19, 10, 2, 17, 5, 15, 13, 18, 11, 2, 5, 2, 8, 5, 13, 23, 29, 7, 12, 5, 13, 18, 2, 27, 19, 25, 16, 8, 2, 6, 9, 2, 27, 19, 22, 24, 12, 2, 24, 12, 9, 2, 24, 22, 19, 25, 6, 16, 9, 2, 19, 10, 2, 11, 9, 24, 24, 13, 18, 11, 2, 25, 20, 2, 5, 18, 8, 2, 2, 2, 2, 2, 2]
Pattern length is 73
Seed:
" pg0nbljoh0b0ebjtzdibjo0xpvme0cf0xpsui0uif0uspvcmf0pg0hfuujoh0vq0boe000000 "
p
p
p
p
p
p
p
p
p
p

Done.
